## Chapter 32: Traveling Salesperson and Shortest Path problems

In [ ]:
using LightGraphs, SimpleWeightedGraphs, Plots, GraphRecipes, Combinatorics, HTTP, JSON

The Traveling Salesperson Problem (TSP) is based on an example in which a salesperson must travel to visit a set of cities and return home.  The goal is to find the shortest path (could be shortest distance or time) for the entire trip.

Mathematically, this problem is thought of as a graph.  Consider the following: 

In [ ]:
g = SimpleWeightedGraph(5)
add_edge!(g, 1, 2, 13)
add_edge!(g, 1, 3, 15)
add_edge!(g, 1, 4, 9)
add_edge!(g, 1, 5, 13)
add_edge!(g, 2, 3, 8)
add_edge!(g, 2, 4, 12)
add_edge!(g, 2, 5, 20)
add_edge!(g, 3, 4, 17)
add_edge!(g, 3, 5, 15)
add_edge!(g, 4, 5, 18)

edge_labels = Dict()
dmatrix=zeros(Float64,5,5)
for i=1:5
  for j=1:5
    dmatrix[i,j]= get_weight(g,i,j)
    if dmatrix[i,j] > 0 
      edge_labels[(i,j)] = dmatrix[i,j]
    end
  end
end

In [ ]:
graphplot(g,names=1:5,edgelabel=edge_labels)

Imagine that the salesperson is at any one of the node (vertices) and needs to visit any node.  For example, consider the path `1-->2-->3-->4-->5-->1` which has a total distance of $13+8+17+18+13=69$. 

We'll solve this by exhaustive search.  For a TSP problem with $n$ cities, there are a total of $(n-1)!$ paths to check.

First, an array of all of the weights (distances) of the graph was generated above:

In [ ]:
dmatrix

where the distance between nodes $i$ and $j$ is in the array at both row $i$, column $j$ and row $i$, column $j$.

We can then use a path to sum up the distances.  Here's a function to do that:

In [ ]:
function totalDistance(path::Vector{T1}, distances::Array{T2,2}) where {T1 <: Real, T2 <: Real}
    local dist = distances[path[end],path[1]]
    for i=1:length(path)-1
        dist += distances[path[i],path[i+1]]
    end
    dist
end     

We can find the distance of the path $1\rightarrow2\rightarrow3\rightarrow4\rightarrow5\rightarrow1$ with

In [ ]:
totalDistance([1,2,3,4,5],dmatrix)

We can find the distance of the path $2\rightarrow1\rightarrow4\rightarrow3\rightarrow5\rightarrow2$ with

In [ ]:
totalDistance([2,1,4,3,5],dmatrix)

One way to find the path with the shortest distance is to go through all of the possible paths, which is a permutation of the vector `[1,2,3,4,5]`.  The function `nthperm` returns a permutation of the first argument (an array) and the integer $n$.  This function is in the `Combinatorics` package.  Here's it applied to an array of strings:

In [ ]:
for i=1:24
  p = nthperm(["horse","zebra","camel","whale"],i)
  @show p
end

In [ ]:
nthperm(1:5,2)

In [ ]:
nthperm(1:5,30)

To find the shortest path by brute-force calculation:

In [ ]:
d=map(k->totalDistance(nthperm(1:5,k),dmatrix),1:factorial(5))

In [ ]:
findmin(d)

This shows that the minimum is 57 and is located at index 13.  The order of nodes is: 

In [ ]:
nthperm(1:5,findmin(d)[2])

This shows that the minimum path starting at 1 is $1\rightarrow4\rightarrow2\rightarrow3\rightarrow5\rightarrow1$ has a total distance of 57

#### Minimizing distances between cities:

We showed earlier how to fetch distances between a set of cities.  Here's that code again

In [ ]:
mapquest_key = "WL9lkNndull1hszCbog4MicH6ZRPGEEO" ## add your key here. 

In [ ]:
towns = Dict(
    "locations"=>["Fitchburg, MA","Boston, MA","Providence, RI","Nashua, NH","Portland, ME", "Concord, NH"],
      "options" => Dict("allToAll"=>true))

In [ ]:
req2 = HTTP.request("POST","http://www.mapquestapi.com/directions/v2/routematrix?key=$mapquest_key",[],JSON.json(towns));

In [ ]:
distance_dict = JSON.parse(String(req2.body))
dist_array = collect(transpose(reshape(collect(Iterators.flatten(distance_dict["distance"])),(6,6))))

Now let's find all routes through the towns:

In [ ]:
d=map(k->totalDistance(nthperm(1:6,k),dist_array),1:factorial(5));

In [ ]:
findmin(d)

In [ ]:
nthperm(1:6,71)

The cities in order are:

In [ ]:
towns["locations"][nthperm(1:6,71)]

### Simulated Annealing

As the number of towns increase, the brute force method of finding all routes is not possible.  This grows as a factorial. Alternatively, we will use randomness to solve this problem.  Here's the idea:

1. Start with any route.  Calculate the distance of the route
2. randomly swap two towns in the route and calculate the distance. 
3. If the distance decreases, replace the route with the new route and the distance with the new distance. 
4. Repeat steps 2 and 3 as many times as possible.

In [ ]:
route = collect(1:6)

In [ ]:
dist = totalDistance(route, dist_array)

In [ ]:
i=rand(1:6)
j=rand(1:6)

In [ ]:
route[[i,j]] = route[[j,i]]

In [ ]:
new_dist = totalDistance(route,dist_array)

In [ ]:
function TSP_simulated_annealing(distances::Array{T,2},iter::Int) where T<: Real
    ###This will run simulated annealing on the TSP and return the shortest distance after iter iterations
 
    local N = size(distances,1)
    local path = collect(1:N)
    local min_dist = totalDistance(path,distances)
    local min_path = copy(path)  ## you need to do the copy to make a copy of the contents.
  
    for i=1:iter
        k=rand(1:N)
        j=rand(1:N)
        path[[j,k]]=path[[k,j]]  ## swap the elements at j and k
        dist=totalDistance(path,distances)
        if dist<min_dist
            min_dist = dist
            min_path = copy(path)
        end
    end
    (min_path,min_dist)
end

In [ ]:
soln = TSP_simulated_annealing(dist_array,100_000)

In [ ]:
towns["locations"][soln[1]]

#### Let's say we have more towns:

The following will generate some fake data for distances:

In [ ]:
A = 25*ones(10,10)+25*rand(10,10)
dist10 = A + transpose(A);
for i=1:10
  dist10[i,i]= 0 
end
dist10

In [ ]:
find_TSP(dist10,10_000_000)

### Shortest Path Problems:

Similar to the traveling salesperson problem, a shortest path problem also occurs on a weighted graph.  Consider the following: 

In [ ]:
g = SimpleWeightedGraph(6)
add_edge!(g, 1, 2, 9)
add_edge!(g, 1, 4, 11)
add_edge!(g, 1, 5, 16)
add_edge!(g, 2, 3, 15)
add_edge!(g, 2, 5, 4)
add_edge!(g, 3, 5, 10)
add_edge!(g, 3, 6, 6)
add_edge!(g, 4, 5, 20)
add_edge!(g, 5, 6, 17)

edge_labels = Dict()
wts=zeros(Float64,6,6)
for i=1:6
  for j=1:6
    wts[i,j]= get_weight(g,i,j)
    if wts[i,j] > 0 
      edge_labels[(i,j)] = wts[i,j]
    end
  end
end

We can plot the graph with: 

In [ ]:
graphplot(g,names=["A","B","C","D","E","F"],edgelabel=edge_labels)

In a shortest path problem, we wish to go from one node to another one.  This is actually an easier problem to solve that the traveling salesperson problem.  There are a few standard algorithms including [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) and [$A^{\star}$ search algorithm](https://en.wikipedia.org/wiki/A%2A_search_algorithm)

Here's the shortest path from node 1 to 3 (A to C):

In [ ]:
enumerate_paths(dijkstra_shortest_paths(g, 1), 3)

In [ ]:
enumerate_paths(dijkstra_shortest_paths(g,5),6)

In [ ]:
enumerate_paths(dijkstra_shortest_paths(g, 1), 5)

In [ ]:
enumerate_paths(dijkstra_shortest_paths(g, 1), 6)

If needed, you can find the shortest path from any node:

In [ ]:
enumerate_paths(dijkstra_shortest_paths(g,5))

This algorithm can be combined with real distances between points in a city (say places or intersections) to determine shortest paths similar to mapping apps like google maps or Apple Maps. 